# LoL Stats Visualization

In [1]:
import backend.downloader as dl
import backend.assets as assets
import backend.db as db
import pandas as pd
import altair as alt

## Download data

In [2]:
SUMMONERS = ["Helikk", "marcosalome", "smallfatant", "Vempramim", "Raketes", "Raketex", "MadameTripas"]
dl.get_data(SUMMONERS)

DB connection created.
Downloaded Summoner profile(s).
Downloaded 779 match ids for 'Helikk'.
Downloaded 1471 match ids for 'Marcosalome'.
Downloaded 1279 match ids for 'smallfatant'.
Downloaded 601 match ids for 'Vempramim'.
Downloaded 3224 match ids for 'Raketes'.
Downloaded 316 match ids for 'Raketex'.
Downloaded 1945 match ids for 'MadameTripas'.
Total downloaded match ids: 9615.
Read 7816 lines from 'match_details' table in DB.
Compared DB with downloaded data: 35 new matches found.
Getting data on 35 matches.
name 'ApiError' is not defined
Error in match #34 (id: <4955629649)>.
Finished processing 35 matches.
Data on 35 matches downloaded.
DB connection closed.


In [13]:
conn = db.create_connection("./backend/db/normalized.db")
md = db.read_full_table(conn, "match_details")
t = db.read_full_table(conn, "teams")
p = db.read_full_table(conn, "participants")
pi = db.read_full_table(conn, "participant_identities")
db.close_connection(conn)

DB connection created.
Read 7851 lines from 'match_details' table in DB.
Read 15702 lines from 'teams' table in DB.
Read 78502 lines from 'participants' table in DB.
Read 78502 lines from 'participant_identities' table in DB.
DB connection closed.


In [14]:
aconn = db.create_connection("./backend/db/assets.sqlite")
c = db.read_full_table(aconn, "champions")
q = db.read_full_table(aconn, "queues")
db.close_connection(aconn)

DB connection created.
Read 152 lines from 'champions' table in DB.
Read 80 lines from 'queues' table in DB.
DB connection closed.


In [15]:
df = pd.merge(p, c, on=["championId"])
df = pd.merge(df, md, on=["gameId"])
df = pd.merge(df, q, on=["queueId"])
df = pd.merge(df, pi, on=["gameId", "participantId"])
df.columns = df.columns.str.replace(".", "_")

In [16]:
df["datetime"] = pd.to_datetime(df["gameCreation"], unit="ms")
df

,participantId,teamId,championId,spell1Id,spell2Id,gameId,stats_participantId,stats_win,stats_item0,stats_item1,...,queue,player_platformId,player_accountId,player_summonerName,player_summonerId,player_currentPlatformId,player_currentAccountId,player_matchHistoryUri,player_profileIcon,datetime
0,1,100,99,4,12,4953382523,1,1,3041,1058,...,Co-op vs. AI Intermediate Bot games,EUW1,DI0sm3UglRUWDjJtYB434V2-mfYzY6TkwUfHs0qKcEsQkNw,Astro Defense,F7SxJvnJxwkPuPFVoVUdLrd9h1C9Li3bA4bjrMouSjpS_us,EUW1,DI0sm3UglRUWDjJtYB434V2-mfYzY6TkwUfHs0qKcEsQkNw,/v1/stats/player_history/EUW1/204426654,4508,2020-11-29 00:11:23.261
1,2,100,121,11,4,4953382523,2,1,6691,2031,...,Co-op vs. AI Intermediate Bot games,EUW1,mcyBQoNC7cSKWUejhRSSMCRetMNHYWgTR8M64_dXkkT3gP...,Tagrosso0,2aVKkVj8F7IcV_-g9FBoL3oOKQWBaJq4Yc-6VWWTMMtFXF...,EUW1,mcyBQoNC7cSKWUejhRSSMCRetMNHYWgTR8M64_dXkkT3gP...,/v1/stats/player_history/EUW1/2598048272311488,1,2020-11-29 00:11:23.261
2,3,100,12,4,12,4953382523,3,1,1054,3152,...,Co-op vs. AI Intermediate Bot games,EUW1,EJQfVTIvnTdaam-Lcn1yp9r2X7QJc1NlOwz5Xj60oVEPNw,smallfatant,apc74Qh1EZ9YEIOU9ArIeown83KzV3Ebv5mi717QyMlyV7Y,EUW1,EJQfVTIvnTdaam-Lcn1yp9r2X7QJc1NlOwz5Xj60oVEPNw,/v1/stats/player_history/EUW1/39539715,4607,2020-11-29 00:11:23.261
3,10,200,12,0,0,4953382523,10,0,3860,3009,...,Co-op vs. AI Intermediate Bot games,,0,Alistar,None,EUW1,0,/v1/stats/player_history//0,0,2020-11-29 00:11:23.261
4,4,100,412,14,4,4953382523,4,1,1055,6671,...,Co-op vs. AI Intermediate Bot games,EUW1,-ZpYmpIz2Kqfy84WUgs8i7WPL1iGr37Qrjl3EAbvMTbQPBc,Helikk,4m6VdaCefu0AXMPmPpXg7RjVqRb-OiEPenLGmSoju0lHjgE,EUW1,-ZpYmpIz2Kqfy84WUgs8i7WPL1iGr37Qrjl3EAbvMTbQPBc,/v1/stats/player_history/EUW1/203411224,1437,2020-11-29 00:11:23.261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78367,10,200,89,0,0,4942070868,10,0,3860,3009,...,Co-op vs. AI Beginner Bot games,,0,Leona,None,EUW1,0,/v1/stats/player_history//0,0,2020-11-22 13:34:51.578
78368,4,100,23,4,12,4942070868,4,1,1038,6672,...,Co-op vs. AI Beginner Bot games,EUW1,EJQfVTIvnTdaam-Lcn1yp9r2X7QJc1NlOwz5Xj60oVEPNw,smallfatant,apc74Qh1EZ9YEIOU9ArIeown83KzV3Ebv5mi717QyMlyV7Y,EUW1,EJQfVTIvnTdaam-Lcn1yp9r2X7QJc1NlOwz5Xj60oVEPNw,/v1/stats/player_history/EUW1/39539715,4607,2020-11-22 13:34:51.578
78369,5,100,518,3,12,4942070868,5,1,2423,3111,...,Co-op vs. AI Beginner Bot games,EUW1,x5ERY4ihcqvi8Cx91WdESrZ9paxdmNpClSP5Lx6Mi_VM-g...,pjigc,n0Q3VsDGRZzkMFWvQeDpThhO098fJi0Pge7ThCTahEFI0w...,EUW1,x5ERY4ihcqvi8Cx91WdESrZ9paxdmNpClSP5Lx6Mi_VM-g...,/v1/stats/player_history/EUW1/2567624771396928,3545,2020-11-22 13:34:51.578
78370,7,200,19,0,0,4942070868,7,0,1055,1001,...,Co-op vs. AI Beginner Bot games,,0,Warwick,None,EUW1,0,/v1/stats/player_history//0,0,2020-11-22 13:34:51.578


In [17]:
df.to_csv("./backend/outputs/denormalized_data.csv")

## Get data from DBs

In [ ]:
# Read from DB to dataframe
NORMALIZED_DB_PATH = "./backend/db/data_normalized.db"

nconn = db.create_connection(NORMALIZED_DB_PATH)
md = db.read_full_table(nconn, "match_details")
t = db.read_full_table(nconn, "teams")
p = db.read_full_table(nconn, "participants")
pi = db.read_full_table(nconn, "participant_identities")
db.close_connection(nconn)

In [ ]:
# Add queue and map info
ASSETS_PATH = "./backend/db/assets.db"

aconn = db.create_connection(ASSETS_PATH)
qs = db.read_full_table(aconn, "queues")
db.close_connection(aconn)

## Match details exploration
### Create additional variables 

In [ ]:
# Get date from gameCreation
md["datetime"] = pd.to_datetime(md["gameCreation"], unit="ms")

# Get approximate match duration
md["gameDurationMinutes"] = round(md["gameDuration"] / 60, 0)
md["gameDurationMinutes"] = md["gameDurationMinutes"].astype("int16")

# Add queue names - NEXUSBLITZ games may be lost as their queue seems to not exists in the queues dataset
if not "queue" in md.columns:
    md = pd.merge(md, qs, on="queueId")

md

### Basic count charts

#### gameMode

In [ ]:
common_args = [
    alt.X("count():Q", axis=alt.Axis(grid=True, ticks=True)),
]

mode_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Text("count():Q"),
)

mode_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameMode:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(mode_counts, mode_text, data=md)

In [ ]:
alt.layer(mode_counts, mode_text, data=md).facet(
    row=alt.Row("datetime:N", timeUnit="yearmonth")
)

In [ ]:
alt.Chart(md).mark_tick(width=40, thickness=3).encode(
    alt.X("gameMode:N"),
    alt.Y("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    tooltip=[alt.Tooltip("count():Q"), alt.Tooltip("gameDurationMinutes:Q")]
).configure_view(width=350, height=350)

In [ ]:
alt.Chart(md).transform_density(
    'gameDurationMinutes',
    as_=['gameDurationMinutes', 'density'],
    groupby=['gameMode']
).mark_area(orient='horizontal').encode(
    y='gameDurationMinutes:Q',
    color='gameMode:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'gameMode:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
        ),
    )
).configure_facet(
    spacing=0
)

#### gameDurationMinutes

In [ ]:
duration_text = alt.Chart().mark_text(dx=10).encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Text("count():Q"),
)

duration_counts = alt.Chart().mark_bar().encode(
    *common_args,
    alt.Y("gameDurationMinutes:N"),
    alt.Color("datetime:N", timeUnit="year"),
    alt.Tooltip("count():Q")
)

alt.layer(duration_counts, duration_text, data=md)

In [ ]:
duration_counts = duration_counts.encode(
    alt.Color("gameMode:N")
)

alt.layer(duration_counts, duration_text, data=md)

#### datetime

In [ ]:
def datetime_layer_chart(time_unit):
    datetime_text = alt.Chart().mark_text(dx=10).encode(
        *common_args,
        alt.Text("count():Q"),
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    datetime_counts = alt.Chart().mark_bar().encode(
        *common_args,
        alt.Y("datetime:N", timeUnit=time_unit)
    )

    return alt.layer(datetime_counts, datetime_text, data=md)

datetime_layer_chart("year")

In [ ]:
datetime_layer_chart("yearmonth")

In [ ]:
datetime_layer_chart("day")

In [ ]:
datetime_layer_chart("hours")

In [ ]:
alt.Chart(md).mark_circle().encode(
    alt.X("datetime:T", timeUnit="hoursminutes"),
    alt.Y("gameDurationMinutes:Q"),
    #alt.Color("datetime:N", timeUnit="day"),
    #alt.Size("count()"),
    tooltip=[alt.Tooltip("datetime:T", timeUnit="hoursminutes"),
        alt.Tooltip("gameDurationMinutes:Q")],
)

In [ ]:
hm = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Color("count()")
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:O", timeUnit="day"),
    alt.Y("gameMode:O"),
    alt.Text("count()")
)

(hm + text).configure_view(width=400, height=200)

### Maps

In [ ]:
alt.Chart(md).mark_bar().encode(
    alt.X("map:N"),
    alt.Y("count():Q")
).configure_view(width=200)

In [ ]:
lines = alt.Chart(md).mark_line(point=True).encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("count():Q"),
    alt.Color("map:N"),
    alt.Tooltip("count():Q")
) 
bars = alt.Chart(md).transform_calculate(diff="datum.map == 'Howling Abyss' ? 1 : -1").mark_bar().encode(
    alt.X("datetime:T", timeUnit="yearmonth"),
    alt.Y("sum(diff):Q"),
    alt.Tooltip("sum(diff):Q")
)

lines & bars

#### Queues

In [ ]:
alt.Chart(md).mark_circle(size=50).encode(
    alt.Y("queue:N"),
    alt.X("gameDurationMinutes:Q"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q")
).configure_view(height=300, width=500)

In [ ]:
squares = alt.Chart(md).mark_rect().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Color("count():Q"),
    alt.Tooltip("count():Q"),
)

text = alt.Chart(md).mark_text().encode(
    alt.X("datetime:N", timeUnit="day"),
    alt.Y("queue:N"),
    alt.Text("count():Q")
)

(squares + text).configure_view(height=300, width=350)

## Participants exploration

In [ ]:
TBC